Verisetinin indirilmesi

In [ ]:
# ↳ yaklaşık 3 GB .xz dosyası (~11 GB açılmış metin)
!wget -c http://data.statmt.org/cc-100/tr.txt.xz \
      -O /content/drive/MyDrive/turkish_llm/cc100-tr.txt.xz


--2025-08-07 16:17:41--  http://data.statmt.org/cc-100/tr.txt.xz
Resolving data.statmt.org (data.statmt.org)... 129.215.32.28
Connecting to data.statmt.org (data.statmt.org)|129.215.32.28|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://data.statmt.org/cc-100/tr.txt.xz [following]
--2025-08-07 16:17:41--  https://data.statmt.org/cc-100/tr.txt.xz
Connecting to data.statmt.org (data.statmt.org)|129.215.32.28|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5710467188 (5.3G) [application/x-xz]
Saving to: ‘/content/drive/MyDrive/turkish_llm/cc100-tr.txt.xz’

/content/drive/MyDr 100%[===================>]   5.32G  9.61MB/s    in 9m 35s  

2025-08-07 16:27:16 (9.48 MB/s) - ‘/content/drive/MyDrive/turkish_llm/cc100-tr.txt.xz’ saved [5710467188/5710467188]



In [ ]:
!pip install --quiet fasttext tqdm xxhash


In [ ]:
import pathlib, requests, shutil, os, gzip, lzma, xxhash, json, math, itertools
from tqdm import tqdm

lid_path = pathlib.Path('/content/lid.176.bin')
if not lid_path.exists():
    url = 'https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin'
    print('İndiriliyor… (~126 MB)')
    with requests.get(url, stream=True) as r, open(lid_path, 'wb') as f:
        shutil.copyfileobj(r.raw, f)
    print('✓ Dil modeli indirildi →', lid_path)

import fasttext
lid_model = fasttext.load_model(str(lid_path))


İndiriliyor… (~126 MB)
✓ Dil modeli indirildi → /content/lid.176.bin


In [ ]:
!pip install --quiet --upgrade "numpy<2.0" "fasttext==0.9.2"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 43.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [ ]:
# === CC100-tr TEMİZLEME HÜCRESİ =========================================
# Girdi  : /content/drive/MyDrive/turkish_llm/cc100-tr.txt.xz
# Çıktı  : /content/drive/MyDrive/turkish_llm/cc100-tr.clean.txt.gz
# Amaç   : Dil filtresi (TR), uzun/kısa satır filtresi, deduplikasyon,
#          ~2B token hedefi (isteğe bağlı) – çıkış ≈ 12 GB ham metin

from google.colab import drive
drive.mount('/content/drive', force_remount=False)   # zaten bağlıysa değişmez

import lzma, gzip, xxhash, fasttext, pathlib, requests, shutil
from tqdm import tqdm

# -------- Ayarlar --------
RAW_PATH   = '/content/drive/MyDrive/turkish_llm/cc100-tr.txt.xz'
CLEAN_PATH = '/content/drive/MyDrive/turkish_llm/cc100-tr.clean.txt.gz'

MIN_TOKENS    = 5          # satırdaki min token
MAX_TOKENS    = 200        # satırdaki max token
MAX_CHARS     = 1000       # satırdaki max karakter
TARGET_TOKENS = 2_000_000_000  # None => sınır yok (~2B token hedefi)

# -------- Dil modeli (FastText LID-176) --------
lid_path = pathlib.Path('/content/lid.176.bin')
if not lid_path.exists():
    print('► FastText model indiriliyor…')
    url = 'https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin'
    with requests.get(url, stream=True) as r, open(lid_path, 'wb') as f:
        shutil.copyfileobj(r.raw, f)
ft = fasttext.load_model(str(lid_path))

def is_turkish(text: str) -> bool:
    """0-1 Türkçe satır filtresi (≥0.80 olasılık)."""
    text = text.strip()
    if not text:
        return False
    label, prob = ft.predict(text, k=1)
    return label[0] == '__label__tr' and prob[0] > 0.80

def simple_tokenize(t: str):
    return t.split()

# -------- Akışlı okuma / yazma --------
seen_hashes = set()
total_tokens = 0

with lzma.open(RAW_PATH, mode='rt', encoding='utf-8', errors='ignore') as fin, \
     gzip.open(CLEAN_PATH, mode='wt', encoding='utf-8') as fout, \
     tqdm(total=None, unit=' satır', desc='İşleniyor') as pbar:

    for raw_line in fin:
        pbar.update()
        line = raw_line.rstrip('\n')

        # Uzunluk filtreleri
        if len(line) < 2 or len(line) > MAX_CHARS:
            continue
        tokens = simple_tokenize(line)
        if not (MIN_TOKENS <= len(tokens) <= MAX_TOKENS):
            continue

        # Dil filtresi
        if not is_turkish(line):
            continue

        # Basit deduplikasyon
        h = xxhash.xxh64(line).intdigest()
        if h in seen_hashes:
            continue
        seen_hashes.add(h)
        if len(seen_hashes) > 10_000_000:   # bellek koruması
            seen_hashes.clear()

        # Yaz & sayaç
        fout.write(line + '\n')
        total_tokens += len(tokens)

        if TARGET_TOKENS and total_tokens >= TARGET_TOKENS:
            print(f'\n⏹  {total_tokens:,} token hedefine ulaşıldı. Durduruluyor…')
            break

print(f'\n✓ Temizleme tamamlandı → {CLEAN_PATH}')
print(f'Toplam token: {total_tokens:,}')
# ========================================================================


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


İşleniyor: 99822767 satır [2:55:03, 9503.42 satır/s] 


⏹  2,000,000,047 token hedefine ulaşıldı. Durduruluyor…

✓ Temizleme tamamlandı → /content/drive/MyDrive/turkish_llm/cc100-tr.clean.txt.gz
Toplam token: 2,000,000,047


In [ ]:
!ls -lh /content/drive/MyDrive/turkish_llm/cc100-tr.clean.txt.gz
!gzip -l  /content/drive/MyDrive/turkish_llm/cc100-tr.clean.txt.gz


-rw------- 1 root root 5.9G Aug  7 19:32 /content/drive/MyDrive/turkish_llm/cc100-tr.clean.txt.gz
         compressed        uncompressed  ratio uncompressed_name
         6273266558          3584799583 -75.0% /content/drive/MyDrive/turkish_llm/cc100-tr.clean.txt


In [1]:
# === HÜCRE 1: İMPORTLAR VE GPU KONTROLÜ ===================================

import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import numpy as np
import sentencepiece as spm
import gzip
import time
import os
import gc
from dataclasses import dataclass
from typing import Optional, Tuple
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

# GPU kontrolü
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("GPU bulunamadı!")

# Pathler
DATA_PATH = '/content/drive/MyDrive/turkish_llm/cc100-tr.clean.txt.gz'
TOKENIZER_PATH = '/content/drive/MyDrive/turkish_llm/tokenizer_tr_32k_v2.model'
CHECKPOINT_DIR = '/content/drive/MyDrive/turkish_llm/checkpoints/'
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

print("İmportlar tamamlandı")

GPU: NVIDIA A100-SXM4-40GB
VRAM: 42.5 GB
İmportlar tamamlandı


In [2]:
# === HÜCRE 2: TOKENIZER YÜKLEME ===================================

# Tokenizer yükle
sp = spm.SentencePieceProcessor(model_file=TOKENIZER_PATH)
print(f"Tokenizer yüklendi (vocab size: {sp.vocab_size()})")

# Test
test_text = "Merhaba dünya, nasılsınız?"
tokens = sp.encode(test_text)
decoded = sp.decode(tokens)
print(f"\nTest: '{test_text}'")
print(f"Tokens ({len(tokens)}): {tokens}")
print(f"Decoded: '{decoded}'")

Tokenizer yüklendi (vocab size: 32000)

Test: 'Merhaba dünya, nasılsınız?'
Tokens (6): [2514, 1212, 31895, 782, 1243, 31934]
Decoded: 'Merhaba dünya, nasılsınız?'


In [3]:
# === HÜCRE 3: MODEL KONFİGÜRASYONU ===================================

@dataclass
class ModelConfig:
    dim: int = 768
    n_layers: int = 12
    n_heads: int = 12
    n_kv_heads: int = 12
    vocab_size: int = 32000
    max_seq_len: int = 1024  # Optimize edilmiş
    dropout: float = 0.0

    multiple_of: int = 256
    ffn_dim_multiplier: float = 2.7
    norm_eps: float = 1e-5
    rope_theta: float = 10000.0

    def __post_init__(self):
        hidden_dim = int(2 * self.dim * self.ffn_dim_multiplier / 3)
        self.ffn_hidden_dim = self.multiple_of * ((hidden_dim + self.multiple_of - 1) // self.multiple_of)
        self.head_dim = self.dim // self.n_heads

@dataclass
class TrainingConfig:
    batch_size: int = 8           # A100 için optimize
    max_length: int = 1024
    num_workers: int = 2

    learning_rate: float = 5e-4
    weight_decay: float = 0.1
    adam_beta1: float = 0.9
    adam_beta2: float = 0.95
    grad_clip: float = 1.0

    warmup_steps: int = 1000
    max_steps: int = 10000
    eval_interval: int = 250
    save_interval: int = 1000
    log_interval: int = 10

    use_amp: bool = True

config = ModelConfig()
train_config = TrainingConfig()

print(f"Model Config: {config.n_layers} layers, {config.dim} dim, {config.n_heads} heads")
print(f"Training Config: batch_size={train_config.batch_size}, max_steps={train_config.max_steps}")

Model Config: 12 layers, 768 dim, 12 heads
Training Config: batch_size=8, max_steps=10000


In [4]:
# === HÜCRE 4: MODEL COMPONENTLERİ ===================================

# RMSNorm
class RMSNorm(nn.Module):
    def __init__(self, dim: int, eps: float = 1e-6):
        super().__init__()
        self.eps = eps
        self.weight = nn.Parameter(torch.ones(dim))

    def forward(self, x):
        norm = x * torch.rsqrt(x.pow(2).mean(-1, keepdim=True) + self.eps)
        return self.weight * norm

# SwiGLU activation
class SwiGLU(nn.Module):
    def forward(self, x, gate):
        return F.silu(gate) * x

# FeedForward
class FeedForward(nn.Module):
    def __init__(self, config: ModelConfig):
        super().__init__()
        self.w1 = nn.Linear(config.dim, config.ffn_hidden_dim, bias=False)
        self.w2 = nn.Linear(config.ffn_hidden_dim, config.dim, bias=False)
        self.w3 = nn.Linear(config.dim, config.ffn_hidden_dim, bias=False)
        self.swiglu = SwiGLU()

    def forward(self, x):
        return self.w2(self.swiglu(self.w1(x), self.w3(x)))

print("Model componentleri tanımlandı")

Model componentleri tanımlandı


In [5]:
# === HÜCRE 5: ROPE VE ATTENTION ===================================

# RoPE fonksiyonları
def precompute_freqs_cis(dim: int, max_seq_len: int, theta: float = 10000.0):
    freqs = 1.0 / (theta ** (torch.arange(0, dim, 2)[: (dim // 2)].float() / dim))
    t = torch.arange(max_seq_len, device=freqs.device)
    freqs = torch.outer(t, freqs).float()
    freqs_cis = torch.polar(torch.ones_like(freqs), freqs)
    return freqs_cis

def reshape_for_broadcast(freqs_cis: torch.Tensor, x: torch.Tensor):
    ndim = x.ndim
    assert 0 <= 1 < ndim
    assert freqs_cis.shape == (x.shape[1], x.shape[-1])
    shape = [d if i == 1 or i == ndim - 1 else 1 for i, d in enumerate(x.shape)]
    return freqs_cis.view(*shape)

def apply_rotary_emb(xq, xk, freqs_cis):
    xq_ = torch.view_as_complex(xq.float().reshape(*xq.shape[:-1], -1, 2))
    xk_ = torch.view_as_complex(xk.float().reshape(*xk.shape[:-1], -1, 2))
    freqs_cis = reshape_for_broadcast(freqs_cis, xq_)
    xq_out = torch.view_as_real(xq_ * freqs_cis).flatten(3)
    xk_out = torch.view_as_real(xk_ * freqs_cis).flatten(3)
    return xq_out.type_as(xq), xk_out.type_as(xk)

# Attention
class Attention(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.n_heads = config.n_heads
        self.n_kv_heads = config.n_kv_heads
        self.head_dim = config.head_dim
        self.n_rep = self.n_heads // self.n_kv_heads

        self.wq = nn.Linear(config.dim, config.n_heads * config.head_dim, bias=False)
        self.wk = nn.Linear(config.dim, config.n_kv_heads * config.head_dim, bias=False)
        self.wv = nn.Linear(config.dim, config.n_kv_heads * config.head_dim, bias=False)
        self.wo = nn.Linear(config.n_heads * config.head_dim, config.dim, bias=False)
        self.dropout = nn.Dropout(config.dropout)

    def forward(self, x, freqs_cis, mask=None):
        bsz, seqlen, _ = x.shape

        xq, xk, xv = self.wq(x), self.wk(x), self.wv(x)
        xq = xq.view(bsz, seqlen, self.n_heads, self.head_dim)
        xk = xk.view(bsz, seqlen, self.n_kv_heads, self.head_dim)
        xv = xv.view(bsz, seqlen, self.n_kv_heads, self.head_dim)

        xq, xk = apply_rotary_emb(xq, xk, freqs_cis)

        if self.n_rep > 1:
            xk = xk.unsqueeze(3).expand(-1, -1, -1, self.n_rep, -1).flatten(2, 3)
            xv = xv.unsqueeze(3).expand(-1, -1, -1, self.n_rep, -1).flatten(2, 3)

        xq = xq.transpose(1, 2)
        xk = xk.transpose(1, 2)
        xv = xv.transpose(1, 2)

        scores = torch.matmul(xq, xk.transpose(2, 3)) / math.sqrt(self.head_dim)
        if mask is not None:
            scores = scores + mask
        scores = F.softmax(scores.float(), dim=-1).type_as(xq)
        scores = self.dropout(scores)
        output = torch.matmul(scores, xv)

        output = output.transpose(1, 2).contiguous().view(bsz, seqlen, -1)
        return self.wo(output)

print("RoPE ve Attention tanımlandı")

RoPE ve Attention tanımlandı


In [6]:
# === HÜCRE 6: TRANSFORMER BLOCK VE ANA MODEL ===================================

# Transformer Block
class TransformerBlock(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.attention = Attention(config)
        self.feed_forward = FeedForward(config)
        self.attention_norm = RMSNorm(config.dim, eps=config.norm_eps)
        self.ffn_norm = RMSNorm(config.dim, eps=config.norm_eps)

    def forward(self, x, freqs_cis, mask=None):
        h = x + self.attention(self.attention_norm(x), freqs_cis, mask)
        out = h + self.feed_forward(self.ffn_norm(h))
        return out

# Ana Model
class TurkishLLaMA(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.tok_embeddings = nn.Embedding(config.vocab_size, config.dim)
        self.dropout = nn.Dropout(config.dropout)
        self.layers = nn.ModuleList([TransformerBlock(config) for _ in range(config.n_layers)])
        self.norm = RMSNorm(config.dim, eps=config.norm_eps)
        self.output = nn.Linear(config.dim, config.vocab_size, bias=False)

        self.register_buffer("freqs_cis", precompute_freqs_cis(
            config.head_dim, config.max_seq_len * 2, config.rope_theta
        ))

    def forward(self, tokens: torch.Tensor):
        bsz, seqlen = tokens.shape
        h = self.tok_embeddings(tokens)
        h = self.dropout(h)

        mask = torch.full((1, 1, seqlen, seqlen), float("-inf"), device=tokens.device)
        mask = torch.triu(mask, diagonal=1).type_as(h)

        freqs_cis = self.freqs_cis[:seqlen]
        for layer in self.layers:
            h = layer(h, freqs_cis, mask)

        h = self.norm(h)
        output = self.output(h)
        return output

# Model oluştur
model = TurkishLLaMA(config).cuda()
total_params = sum(p.numel() for p in model.parameters())
print(f" Model oluşturuldu: {total_params/1e6:.1f}M parametre")
print(f" VRAM kullanımı: {torch.cuda.memory_allocated()/1e9:.2f} GB")

Model oluşturuldu: 120.0M parametre
VRAM kullanımı: 0.48 GB


In [7]:
# === HÜCRE 7: DATASET SINIFI ===================================

class TextDataset(Dataset):
    def __init__(self, file_path, tokenizer, max_length=1024, max_samples=None):
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.samples = []

        print(" Veri yükleniyor...")
        with gzip.open(file_path, 'rt', encoding='utf-8') as f:
            current_chunk = []
            current_len = 0

            for i, line in enumerate(tqdm(f, desc="Satırlar işleniyor", total=max_samples or 100000)):
                if max_samples and len(self.samples) >= max_samples:
                    break

                tokens = tokenizer.encode(line.strip())
                if not tokens:
                    continue

                if current_len + len(tokens) > max_length:
                    if current_chunk:
                        self.samples.append(current_chunk[:max_length])
                    current_chunk = tokens
                    current_len = len(tokens)
                else:
                    current_chunk.extend(tokens)
                    current_len += len(tokens)

            if current_chunk and len(current_chunk) > 100:
                self.samples.append(current_chunk[:max_length])

        print(f" {len(self.samples):,} örnek hazır")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        tokens = self.samples[idx]

        if len(tokens) < self.max_length:
            tokens = tokens + [self.tokenizer.pad_id()] * (self.max_length - len(tokens))

        input_ids = torch.tensor(tokens[:-1], dtype=torch.long)
        labels = torch.tensor(tokens[1:], dtype=torch.long)

        return input_ids, labels

print(" Dataset sınıfı tanımlandı")

 Dataset sınıfı tanımlandı


In [8]:
# === 10M SATIRLA VERİ YÜKLEME (A100 40GB için optimal) ===================================

import gc
import torch
from tqdm import tqdm
import gzip

# Bellek temizliği
torch.cuda.empty_cache()
gc.collect()

print("10M SATIRLIK BÜYÜK VERİ SETİ HAZIRLANIYOR")

class OptimizedTextDataset(Dataset):
    def __init__(self, file_path, tokenizer, max_length=1024, max_lines=10_000_000):
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.samples = []

        print(f"📚 {max_lines:,} satır okunacak...")

        with gzip.open(file_path, 'rt', encoding='utf-8') as f:
            current_chunk = []
            current_len = 0
            lines_processed = 0

            pbar = tqdm(total=max_lines, desc="Satırlar işleniyor")

            for line in f:
                if lines_processed >= max_lines:
                    break

                lines_processed += 1
                pbar.update(1)

                # Boş veya çok kısa satırları atla
                line = line.strip()
                if len(line) < 30:  # Min 30 karakter
                    continue

                tokens = tokenizer.encode(line)
                if not tokens or len(tokens) < 5:
                    continue

                # Chunk'lara böl
                if current_len + len(tokens) > max_length:
                    if len(current_chunk) > 100:
                        self.samples.append(current_chunk[:max_length])

                        # Her 10K örnekte durum bildir
                        if len(self.samples) % 10000 == 0:
                            pbar.set_postfix({"Örnekler": f"{len(self.samples):,}"})

                    current_chunk = tokens
                    current_len = len(tokens)
                else:
                    current_chunk.extend(tokens)
                    current_len += len(tokens)

            pbar.close()

            # Son chunk
            if len(current_chunk) > 100:
                self.samples.append(current_chunk[:max_length])

        print(f"\n {len(self.samples):,} örnek oluşturuldu")
        print(f" Ortalama: {lines_processed/len(self.samples):.1f} satır/örnek")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        tokens = self.samples[idx]

        if len(tokens) < self.max_length:
            tokens = tokens + [self.tokenizer.pad_id()] * (self.max_length - len(tokens))

        input_ids = torch.tensor(tokens[:-1], dtype=torch.long)
        labels = torch.tensor(tokens[1:], dtype=torch.long)

        return input_ids, labels

# Dataset oluştur
dataset = OptimizedTextDataset(
    DATA_PATH,
    sp,
    max_length=1024,
    max_lines=10_000_000  # 10M satır
)

# A100 için optimal batch size
BATCH_SIZE = 32  # VRAM'e göre artırılabilir

train_loader = DataLoader(
    dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=4,
    pin_memory=True,
    drop_last=True,
    persistent_workers=True
)

# Training config güncelle
train_config.batch_size = BATCH_SIZE
train_config.max_steps = 50_000  # Yeterli step
train_config.eval_interval = 500
train_config.save_interval = 5000
train_config.warmup_steps = 2000

# Optimizer'ı yeniden oluştur
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=train_config.learning_rate,
    weight_decay=train_config.weight_decay,
    betas=(train_config.adam_beta1, train_config.adam_beta2),
    fused=True  # A100 optimizasyonu
)

scaler = torch.amp.GradScaler('cuda')

print(f"\n" + "="*60)
print(f" VERİ SETİ ÖZETİ:")
print(f"  Örnek sayısı: {len(dataset):,}")
print(f"  Toplam token: ~{len(dataset) * 1024 / 1e6:.0f}M")
print(f"  Token/parametre: {(len(dataset) * 1024) / (120 * 1e6):.1f}")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Batch count: {len(train_loader):,}")
print(f"  Tokens per batch: {BATCH_SIZE * 1024:,}")

print(f"\n EĞİTİM PLANI:")
print(f"  Max steps: {train_config.max_steps:,}")
print(f"  Warmup steps: {train_config.warmup_steps:,}")
print(f"  Epoch sayısı: ~{train_config.max_steps / len(train_loader):.1f}")
print(f"  Toplam token (training): ~{train_config.max_steps * BATCH_SIZE * 1024 / 1e9:.1f}B")

print(f"\n BELLEK DURUMU:")
print(f"  Model VRAM: {torch.cuda.memory_allocated()/1e9:.2f} GB")
print(f"  Toplam VRAM: 40 GB")
print(f"  Kullanılabilir: {(40 - torch.cuda.memory_allocated()/1e9):.2f} GB")

print(f"\n TAHMİNİ PERFORMANS:")
print(f"  Beklenen hız: ~200-250K token/saniye")
print(f"  Tahmini süre: {train_config.max_steps * BATCH_SIZE * 1024 / (200_000 * 3600):.1f}-{train_config.max_steps * BATCH_SIZE * 1024 / (250_000 * 3600):.1f} saat")

print(f"\n SİSTEM HAZIR! Eğitim başlatılabilir.")
print("="*60)

 10M SATIRLIK BÜYÜK VERİ SETİ HAZIRLANIYOR

📚 10,000,000 satır okunacak...


Satırlar işleniyor: 100%|██████████| 10000000/10000000 [23:20<00:00, 7140.33it/s, Örnekler=400,000]



404,561 örnek oluşturuldu
Ortalama: 24.7 satır/örnek

📊 VERİ SETİ ÖZETİ:
  Örnek sayısı: 404,561
  Toplam token: ~414M
  Token/parametre: 3.5
  Batch size: 32
  Batch count: 12,642
  Tokens per batch: 32,768

📊 EĞİTİM PLANI:
  Max steps: 50,000
  Warmup steps: 2,000
  Epoch sayısı: ~4.0
  Toplam token (training): ~1.6B

💾 BELLEK DURUMU:
  Model VRAM: 0.48 GB
  Toplam VRAM: 40 GB
  Kullanılabilir: 39.52 GB

⚡ TAHMİNİ PERFORMANS:
  Beklenen hız: ~200-250K token/saniye
  Tahmini süre: 2.3-1.8 saat

SİSTEM HAZIR! Eğitim başlatılabilir.


In [10]:
# === EĞİTİM DÖNGÜSÜ (A100 Optimized) ===================================

import time
import json
from datetime import datetime

# Helper fonksiyonlar
def get_lr(step, config):
    """Cosine learning rate schedule with warmup"""
    if step < config.warmup_steps:
        return config.learning_rate * step / config.warmup_steps
    progress = (step - config.warmup_steps) / (config.max_steps - config.warmup_steps)
    return config.learning_rate * 0.5 * (1.0 + np.cos(np.pi * progress))

def format_time(seconds):
    """Saniyeyi saat:dakika:saniye formatına çevir"""
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    seconds = int(seconds % 60)
    return f"{hours:02d}:{minutes:02d}:{seconds:02d}"

# Training step
def train_step(model, batch, optimizer, scaler, config):
    """Tek training step"""
    model.train()

    inputs, targets = batch
    inputs = inputs.cuda()
    targets = targets.cuda()

    # Mixed precision training
    with torch.amp.autocast('cuda', enabled=config.use_amp):
        logits = model(inputs)
        loss = F.cross_entropy(
            logits.view(-1, logits.size(-1)),
            targets.view(-1),
            ignore_index=sp.pad_id()
        )

    # Backward pass
    scaler.scale(loss).backward()

    # Gradient clipping
    scaler.unscale_(optimizer)
    grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), config.grad_clip)

    # Optimizer step
    scaler.step(optimizer)
    scaler.update()
    optimizer.zero_grad(set_to_none=True)

    return loss.item(), grad_norm.item()

# Evaluation
@torch.no_grad()
def evaluate(model, data_loader, num_batches=50):
    """Model evaluation"""
    model.eval()
    losses = []

    for i, (inputs, targets) in enumerate(data_loader):
        if i >= num_batches:
            break

        inputs = inputs.cuda()
        targets = targets.cuda()

        logits = model(inputs)
        loss = F.cross_entropy(
            logits.view(-1, logits.size(-1)),
            targets.view(-1),
            ignore_index=sp.pad_id()
        )
        losses.append(loss.item())

    model.train()
    return np.mean(losses)

# Text generation
@torch.no_grad()
def generate(model, prompt, max_tokens=50, temperature=0.8, top_p=0.9):
    """Generate text from prompt"""
    model.eval()

    # Tokenize prompt
    tokens = sp.encode(prompt)
    tokens = torch.tensor([tokens], dtype=torch.long).cuda()

    generated = []
    for _ in range(max_tokens):
        # Forward pass
        with torch.amp.autocast('cuda', enabled=False):  # Generation'da amp kapalı
            logits = model(tokens)

        logits = logits[:, -1, :] / temperature

        # Top-p (nucleus) sampling
        sorted_logits, sorted_indices = torch.sort(logits, descending=True)
        cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)
        sorted_indices_to_remove = cumulative_probs > top_p
        sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
        sorted_indices_to_remove[..., 0] = 0
        indices_to_remove = sorted_indices_to_remove.scatter(1, sorted_indices, sorted_indices_to_remove)
        logits[indices_to_remove] = float('-inf')

        # Sample
        probs = F.softmax(logits, dim=-1)
        next_token = torch.multinomial(probs, num_samples=1)

        generated.append(next_token.item())

        # Stop at EOS
        if next_token.item() == sp.eos_id():
            break

        # Append token
        tokens = torch.cat([tokens, next_token], dim=1)

        # Truncate if too long
        if tokens.shape[1] > config.max_seq_len:
            tokens = tokens[:, -config.max_seq_len:]

    model.train()
    return sp.decode(generated)

# Ana eğitim fonksiyonu
def train():
    """Ana eğitim döngüsü"""

    print("\n" + "="*70)
    print(" "*25 + "🚀 EĞİTİM BAŞLIYOR!")
    print("="*70)

    # Test prompts
    test_prompts = [
        "Bugün hava",
        "Türkiye'nin başkenti",
        "Yapay zeka teknolojisi",
        "İnsanlar neden",
    ]

    # Training state
    step = 0
    best_loss = float('inf')
    train_losses = []
    eval_losses = []

    # Timing
    start_time = time.time()
    tokens_processed = 0

    # Data iterator
    train_iter = iter(train_loader)

    # Initial evaluation
    print("\n Başlangıç değerlendirmesi...")
    initial_loss = evaluate(model, train_loader, num_batches=50)
    print(f"  Initial loss: {initial_loss:.4f}")
    print(f"  Perplexity: {np.exp(initial_loss):.2f}")

    # Initial generation samples
    print("\n İlk üretim örnekleri:")
    for prompt in test_prompts[:2]:
        output = generate(model, prompt, max_tokens=30, temperature=1.0)
        print(f"  '{prompt}' → {output}")

    print("\n" + "-"*70)
    print("Step |  Loss  |  LR   | Grad | Token/s |  Time  | Progress")
    print("-"*70)

    # Training loop
    while step < train_config.max_steps:
        try:
            batch = next(train_iter)
        except StopIteration:
            train_iter = iter(train_loader)
            batch = next(train_iter)

        # Update learning rate
        lr = get_lr(step, train_config)
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr

        # Training step
        loss, grad_norm = train_step(model, batch, optimizer, scaler, train_config)

        train_losses.append(loss)
        tokens_processed += train_config.batch_size * train_config.max_length
        step += 1

        # Logging
        if step % train_config.log_interval == 0:
            elapsed = time.time() - start_time
            tokens_per_sec = tokens_processed / elapsed
            progress = step / train_config.max_steps * 100

            print(f"{step:5d} | {loss:.4f} | {lr:.1e} | {grad_norm:.2f} | "
                  f"{tokens_per_sec/1000:.1f}K | {format_time(elapsed)} | {progress:5.1f}%")

        # Evaluation
        if step % train_config.eval_interval == 0:
            print("\n" + "="*70)
            print(f" EVALUATION @ Step {step}")
            print("-"*70)

            # Evaluate
            eval_loss = evaluate(model, train_loader, num_batches=100)
            eval_losses.append(eval_loss)

            print(f"  Eval loss: {eval_loss:.4f}")
            print(f"  Perplexity: {np.exp(eval_loss):.2f}")
            print(f"  Train loss (avg): {np.mean(train_losses[-100:]):.4f}")

            # Generation samples
            print("\n   Üretim örnekleri:")
            for i, prompt in enumerate(test_prompts[:3]):
                output = generate(model, prompt, max_tokens=40, temperature=0.8)
                print(f"    [{i+1}] '{prompt}' →")
                print(f"        {output}")

            # Best model
            if eval_loss < best_loss:
                best_loss = eval_loss
                print(f"\n   Yeni en iyi model! (loss: {best_loss:.4f})")

                # Save best model
                best_path = f"{CHECKPOINT_DIR}/best_model.pt"
                torch.save({
                    'step': step,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': best_loss,
                    'config': config,
                }, best_path)

            # Performance stats
            elapsed = time.time() - start_time
            tokens_per_sec = tokens_processed / elapsed
            eta = (train_config.max_steps - step) / (step / elapsed)

            print(f"\n   Performans:")
            print(f"     Tokens/sec: {tokens_per_sec/1000:.1f}K")
            print(f"     Steps/sec: {step/elapsed:.2f}")
            print(f"     ETA: {format_time(eta)}")

            print("="*70 + "\n")
            print("Step |  Loss  |  LR   | Grad | Token/s |  Time  | Progress")
            print("-"*70)

        # Checkpoint save
        if step % train_config.save_interval == 0:
            checkpoint_path = f"{CHECKPOINT_DIR}/checkpoint_step_{step}.pt"
            print(f"\n Checkpoint kaydediliyor: {checkpoint_path}")

            torch.save({
                'step': step,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss,
                'config': config,
                'train_losses': train_losses,
                'eval_losses': eval_losses,
            }, checkpoint_path)

            print(f"   Checkpoint kaydedildi!\n")

        # Early stopping check
        if step > 1000 and loss > 15.0:
            print("\n Model diverge ediyor! Eğitim durduruluyor.")
            break

    # Training complete
    total_time = time.time() - start_time

    print("\n" + "="*70)
    print(" "*25 + " EĞİTİM TAMAMLANDI!")
    print("="*70)
    print(f"\n Final İstatistikler:")
    print(f"  Toplam süre: {format_time(total_time)}")
    print(f"  Toplam step: {step:,}")
    print(f"  Toplam token: {tokens_processed/1e9:.2f}B")
    print(f"  Final loss: {loss:.4f}")
    print(f"  Best eval loss: {best_loss:.4f}")
    print(f"  Ortalama hız: {tokens_processed/(total_time*1000):.1f}K token/s")

    # Save final model
    final_path = f"{CHECKPOINT_DIR}/final_model.pt"
    torch.save({
        'step': step,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
        'config': config,
        'train_losses': train_losses,
        'eval_losses': eval_losses,
        'total_time': total_time,
    }, final_path)

    print(f"\n Final model kaydedildi: {final_path}")

    # Final generation samples
    print("\n Final üretim örnekleri:")
    for prompt in test_prompts:
        output = generate(model, prompt, max_tokens=50, temperature=0.7)
        print(f"\n'{prompt}':")
        print(f"  → {output}")

    return train_losses, eval_losses

# Eğitimi başlat (veri yüklendikten sonra çalıştırın)
print("\n Veri yüklenmesini bekleyin, sonra bu hücreyi çalıştırın:")
print("train_losses, eval_losses = train()")


⏳ Veri yüklenmesini bekleyin, sonra bu hücreyi çalıştırın:
train_losses, eval_losses = train()


In [15]:
# === BELLEK TEMİZLEME VE OPTİMİZASYON ===================================

import gc
import torch

# Belleği tamamen temizle
torch.cuda.empty_cache()
gc.collect()

print("🧹 GPU belleği temizlendi")
print(f" Mevcut VRAM: {torch.cuda.memory_allocated()/1e9:.2f} GB\n")

# Batch size'ı düşür
OPTIMAL_BATCH_SIZE = 8  # 32'den 8'e düşür
GRADIENT_ACCUMULATION = 4  # Efektif batch size: 32

# DataLoader'ı yeniden oluştur
train_loader = DataLoader(
    dataset,
    batch_size=OPTIMAL_BATCH_SIZE,
    shuffle=True,
    num_workers=2,
    pin_memory=True,
    drop_last=True
)

# Config güncelle
train_config.batch_size = OPTIMAL_BATCH_SIZE
train_config.gradient_accumulation = GRADIENT_ACCUMULATION

print(f" Yeni ayarlar:")
print(f"  Batch size: {OPTIMAL_BATCH_SIZE}")
print(f"  Gradient accumulation: {GRADIENT_ACCUMULATION}")
print(f"  Efektif batch size: {OPTIMAL_BATCH_SIZE * GRADIENT_ACCUMULATION}")
print(f"  Batch count: {len(train_loader):,}")

# Optimizer'ı yeniden oluştur
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=train_config.learning_rate,
    weight_decay=train_config.weight_decay,
    betas=(train_config.adam_beta1, train_config.adam_beta2)
)

scaler = torch.amp.GradScaler('cuda')

# PYTORCH_CUDA_ALLOC_CONF ayarla
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

print(f"\n VRAM Durumu:")
print(f"  Kullanılan: {torch.cuda.memory_allocated()/1e9:.2f} GB")
print(f"  Toplam: 40 GB")
print(f"  Boş: {(40 - torch.cuda.memory_allocated()/1e9):.2f} GB")

 GPU belleği temizlendi
 Mevcut VRAM: 1.54 GB

 Yeni ayarlar:
  Batch size: 8
  Gradient accumulation: 4
  Efektif batch size: 32
  Batch count: 50,570

 VRAM Durumu:
  Kullanılan: 1.54 GB
  Toplam: 40 GB
  Boş: 38.46 GB


In [16]:
# === GRADIENT ACCUMULATION'LI TRAINING STEP ===================================

def train_step_with_accumulation(model, data_loader, optimizer, scaler, config, step):
    """Gradient accumulation ile training step"""
    model.train()

    total_loss = 0
    optimizer.zero_grad(set_to_none=True)

    # Gradient accumulation loop
    for micro_step in range(config.gradient_accumulation):
        try:
            batch = next(data_loader)
        except StopIteration:
            data_loader = iter(train_loader)
            batch = next(data_loader)

        inputs, targets = batch
        inputs = inputs.cuda()
        targets = targets.cuda()

        # Forward pass
        with torch.amp.autocast('cuda', enabled=config.use_amp):
            logits = model(inputs)
            loss = F.cross_entropy(
                logits.view(-1, logits.size(-1)),
                targets.view(-1),
                ignore_index=sp.pad_id()
            )
            loss = loss / config.gradient_accumulation  # Scale loss

        # Backward pass
        scaler.scale(loss).backward()
        total_loss += loss.item()

    # Optimizer step (her N micro-step'te bir)
    scaler.unscale_(optimizer)
    grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), config.grad_clip)
    scaler.step(optimizer)
    scaler.update()

    return total_loss * config.gradient_accumulation, grad_norm.item()

# train() fonksiyonunda train_step yerine train_step_with_accumulation kullan
print("\n Sistem optimize edildi!")
print(" Eğitimi tekrar başlatabilirsiniz:")
print("train_losses, eval_losses = train()")


 Sistem optimize edildi!
train_losses, eval_losses = train()


In [17]:
# === EĞİTİMİ BAŞLAT ===================================

# Veri yüklendiğini kontrol et
try:
    print(f" Dataset hazır: {len(dataset):,} örnek")
    print(f" DataLoader hazır: {len(train_loader):,} batch")
    print(f" Model hazır: {sum(p.numel() for p in model.parameters())/1e6:.1f}M parametre")
    print(f" VRAM: {torch.cuda.memory_allocated()/1e9:.2f}/{40:.0f} GB\n")

    # Eğitimi başlat
    train_losses, eval_losses = train()

except NameError as e:
    print(f" Hata: {e}")
    print("Önce veri yükleme hücresinin bitmesini bekleyin!")

Görüntülenen çıkış son 5000 satıra kısaltıldı.
17050 | 4.6403 | 3.9e-04 | 0.89 | 38.6K | 01:00:20 |  34.1%
17060 | 4.6486 | 3.9e-04 | 0.92 | 38.6K | 01:00:22 |  34.1%
17070 | 4.5201 | 3.9e-04 | 0.99 | 38.6K | 01:00:24 |  34.1%
17080 | 4.5719 | 3.9e-04 | 0.93 | 38.6K | 01:00:25 |  34.2%
17090 | 4.7263 | 3.9e-04 | 0.97 | 38.6K | 01:00:27 |  34.2%
17100 | 4.4889 | 3.9e-04 | 0.91 | 38.6K | 01:00:29 |  34.2%
17110 | 4.6243 | 3.9e-04 | 0.92 | 38.6K | 01:00:30 |  34.2%
17120 | 4.6772 | 3.9e-04 | 0.96 | 38.6K | 01:00:32 |  34.2%
17130 | 4.6167 | 3.9e-04 | 0.95 | 38.6K | 01:00:34 |  34.3%
17140 | 4.5441 | 3.9e-04 | 0.91 | 38.6K | 01:00:36 |  34.3%
17150 | 4.6358 | 3.9e-04 | 0.92 | 38.6K | 01:00:37 |  34.3%
17160 | 4.6652 | 3.9e-04 | 0.93 | 38.6K | 01:00:39 |  34.3%
17170 | 4.6051 | 3.9e-04 | 0.91 | 38.6K | 01:00:41 |  34.3%
17180 | 4.6446 | 3.9e-04 | 0.90 | 38.6K | 01:00:42 |  34.4%
17190 | 4.5981 | 3.9e-04 | 0.93 | 38.6K | 01:00:44 |  34.4%
17200 | 4.5814 | 3.9e-04 | 0.91 | 38.6K | 01:00:46 | 

In [21]:
@torch.no_grad()
def generate_improved(model, prompt, max_tokens=50, temperature=0.7, top_p=0.9, repetition_penalty=1.2):
    """Geliştirilmiş generation"""
    model.eval()

    tokens = sp.encode(prompt)
    tokens = torch.tensor([tokens], dtype=torch.long).cuda()
    generated = []

    for _ in range(max_tokens):
        logits = model(tokens)[:, -1, :]

        # Repetition penalty uygula
        for token_id in set(generated[-20:]):  # Son 20 token
            logits[0, token_id] /= repetition_penalty

        # Temperature ve top-p
        logits = logits / temperature
        # ... (top-p sampling)

        next_token = torch.multinomial(F.softmax(logits, dim=-1), 1)
        if next_token.item() == sp.eos_id():
            break
        generated.append(next_token.item())
        tokens = torch.cat([tokens, next_token], dim=1)

    return sp.decode(generated)

# Test
prompts = ["Ben bir", "İstanbul", "Yapay zeka"]
for p in prompts:
    result = generate_improved(model, p, temperature=0.7, repetition_penalty=1.3)
    print(f"'{p}': {result}\n")

'Ben bir': önceki yazımda İstanbul Üniversitesi İşletme Fakültesi’nden mezun oldum. Üniversite, Türkiye’de “Matematik” alanında eğitim almış veya “Türkiye’nin En İyi Öğrenci Derneği” olarak mezun oldum. Buımdaki en büyük başarılardan biri de kuşkusuz “Mate

'İstanbul': ’un en gözde semtlerinden biri olan İzmir, senelerdir de bölgede hayat kurtarıyor. İzmir Büyükşehir Belediyesi’nin sunduğu bayramlaşma programına katılan Başkan Aziz Kocaoğlu, “Şehir merkezimiz her türlü hizmeti vermeye devam ediyor” şeklinde konuştu. İzmir Büyükşehir Belediyesi

'Yapay zeka': ve Cosmic IBM, en çok kullanılan yazılım platformlardan biri olan Y-Bi7 ile birlikte en küçük bir bilgiye sahip olmak için, yalnızca öğrenciler için tasarlanmıştır. Veriler ve bilgiler doğrulayacak şekilde tasarlanmalıdır. Tether



In [22]:
# === MODEL TESTİ ===================================

# Test prompts
test_prompts = [
    "Bugün hava",
    "İstanbul'un",
    "Türkiye ekonomisi",
    "Yapay zeka nedir",
    "En sevdiğim",
    "Bir varmış bir yokmuş",
    "Python programlama dili",
]

print("GENERATION TESTLERİ")
print("="*60)

for prompt in test_prompts:
    print(f"\n Prompt: '{prompt}'")

    # Farklı temperature'larla test
    for temp in [0.5, 0.8]:
        output = generate_improved(
            model,
            prompt,
            max_tokens=40,
            temperature=temp,
            top_p=0.9,
            repetition_penalty=1.3
        )
        print(f"  (temp={temp}): {output}")

print("\n" + "="*60)

# İnteraktif test
print("\n💬 İNTERAKTİF TEST (çıkmak için 'quit' yazın)")
print("-"*60)

while True:
    user_input = input("\n Siz: ")
    if user_input.lower() in ['quit', 'çık', 'exit']:
        break

    response = generate_improved(
        model,
        user_input,
        max_tokens=80,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.25
    )

    print(f" Model: {response}")

 GENERATION TESTLERİ

 Prompt: 'Bugün hava'
  (temp=0.5): , işçi ve emekçi kesimlere göre daha çok sorun çıkarıyor. İşçiler, emekçilerin, emekçilerin, emekçinin ve emekçi kesimlerin sorunları ile mücadele ederken, iş gücü piyasasını da olumsuz
  (temp=0.8): şartları çok kötü değil. Oh değil mi? Sokağımıza girecek oyuncular sanırım yüzde 80 civarında! W Yunusemre’nin en ünlü oyuncusunun sol bek olması, kaleci Meclis Başkanı ve eski futbolcu Metin

 Prompt: 'İstanbul'un'
  (temp=0.5): en büyük ve modern alışveriş merkezi olan İstanbul'un merkezinde yer alan, şehrin binlerce kilometre uzağında bulunan bir restorandır. Şehrin en büyük ve modern alışveriş merkezi olan İstanbul'un merkezinde yer alan, şehrin
  (temp=0.8): en büyük ve popüler alışveriş caddesi Hermessi'nin D&R mağazalarında stoksuz olarak yer alan Parfüm, fiyatları ile alışveriş yapan ve alışveriş yapmak isteyen kişilerin ilk adresi. Parmessa

 Prompt: 'Türkiye ekonomisi'
  (temp=0.5): yüzde 12 ile yüzde 4 arasında değişen bi

In [23]:
import os

# Checkpoint'leri kontrol et
checkpoint_dir = '/content/drive/MyDrive/turkish_llm/checkpoints/'
print(" Kaydedilen checkpoint'ler:")
for file in os.listdir(checkpoint_dir):
    if file.endswith('.pt'):
        size = os.path.getsize(os.path.join(checkpoint_dir, file)) / (1024**3)
        print(f"  ✓ {file} ({size:.2f} GB)")

# Tokenizer'ı kontrol et
tokenizer_path = '/content/drive/MyDrive/turkish_llm/tokenizer_tr_32k_v2.model'
if os.path.exists(tokenizer_path):
    print(f"✓ Tokenizer mevcut: {tokenizer_path}")


 Kaydedilen checkpoint'ler:
  ✓ best_model.pt (1.34 GB)
  ✓ checkpoint_step_5000.pt (1.34 GB)
  ✓ checkpoint_step_10000.pt (1.34 GB)
  ✓ checkpoint_step_15000.pt (1.34 GB)
  ✓ checkpoint_step_20000.pt (1.34 GB)
  ✓ checkpoint_step_25000.pt (1.34 GB)
  ✓ checkpoint_step_30000.pt (1.34 GB)
  ✓ checkpoint_step_35000.pt (1.34 GB)
  ✓ checkpoint_step_40000.pt (1.34 GB)
  ✓ checkpoint_step_45000.pt (1.34 GB)
  ✓ checkpoint_step_50000.pt (1.34 GB)
  ✓ final_model.pt (1.34 GB)
✓ Tokenizer mevcut: /content/drive/MyDrive/turkish_llm/tokenizer_tr_32k_v2.model
